In [20]:
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl 

    100% |████████████████████████████████| 592.3MB 45.3MB/s 


In [21]:
!pip3 install torchvision

    100% |████████████████████████████████| 61kB 2.7MB/s 
    100% |████████████████████████████████| 2.0MB 8.4MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [0]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F

In [0]:
input_size    = 784   # The image size = 28 x 28 = 784
hidden_size   = 500   # The number of nodes at the hidden layer
num_classes   = 10    # The number of output classes. In this case, from 0 to 9
batch_size    = 100   # The size of input data took for one iteration
learning_rate = 1e-2  # The speed of convergence

In [24]:
compose = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((.5, .5, .5), (.5, .5, .5))
        ])
train_dataset = dsets.MNIST(root='./data',
                           train=True,
                           transform=compose,
                           download=True)

test_dataset = dsets.MNIST(root='./data',
                           train=False,
                           transform=compose,
                           download=True)

Processing...
Done!


In [0]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [0]:
def outputSize(in_size, kernel_size, stride, padding):

  output = int((in_size - kernel_size + 2*(padding)) / stride) + 1

  return(output)

In [30]:
outputSize(3,2,2,0)

1

In [0]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Net, self).__init__()                    # Inherited from the parent class nn.Module
        self.fc1 = nn.Linear(input_size, hidden_size)  # 1st Full-Connected Layer: 784 (input data) -> 500 (hidden node)
        self.relu = nn.ReLU()                          # Non-Linear ReLU Layer: max(0,x)
        self.fc2 = nn.Linear(hidden_size, num_classes) # 2nd Full-Connected Layer: 500 (hidden node) -> 10 (output class)
    
    def forward(self, x):                              # Forward pass: stacking each layer together
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [0]:
net = Net(input_size, hidden_size, num_classes)

***SImple CNN Model***

In [0]:
class SimpleCNN(torch.nn.Module):
    
    #Our batch shape for input x is (3, 32, 32)
    
    def __init__(self):
        super(SimpleCNN, self).__init__()
        
        #Input channels = 1, output channels = 128
        #Feauremap Input size = 28, output channels = 14
        self.conv1 = torch.nn.Conv2d(1, 128, kernel_size=3, stride=1, padding=1)
        self.pool = torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        #Input channels = 128, output channels = 256
        #Feauremap Input size = 14, output channels = 7
        self.conv2 = torch.nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.pool2 = torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
         #Input channels = 256, output channels = 128
         #Feauremap Input size = 7, output channels = 3
        self.conv3 = torch.nn.Conv2d(256, 128, kernel_size=3, stride=1, padding=1)
        self.pool3 = torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        #Input channels =128, output channels = 64
        #Feauremap Input size = 3, output channels = 1
        self.conv4 = torch.nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1)
        self.pool4 = torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        #input features, 64 output features (see sizing flow below)
        self.fc1 = torch.nn.Linear(64, 128)
        
        #64 input features, 10 output features for our 10 defined classes
        self.fc2 = torch.nn.Linear(128, 10)
        
    def forward(self, x):
        
        #Computes the activation of the first convolution
        #Size changes from (3, 32, 32) to (18, 32, 32)
        x = F.relu(self.conv1(x))
        
        #Size changes from (18, 32, 32) to (18, 16, 16)
        x = self.pool(x)
        
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        
        x = F.relu(self.conv3(x))
        x = self.pool3(x)
        
        x = F.relu(self.conv4(x))
        x = self.pool4(x)
        
        
        #Reshape data to input to the input layer of the neural net
        #Size changes from (18, 16, 16) to (1, 4608)
        #Recall that the -1 infers this dimension from the other given dimension
        x = x.view(-1, 64)
        
        #Computes the activation of the first fully connected layer
        #Size changes from (1, 4608) to (1, 64)
        x = F.relu(self.fc1(x))
        
        #Computes the second fully connected layer (activation applied later)
        #Size changes from (1, 64) to (1, 10)
        x = self.fc2(x)
        return(x)

In [0]:
sCNN = SimpleCNN()

In [0]:
use_cuda = True

In [0]:
if use_cuda and torch.cuda.is_available():
#     net.cuda()
    sCNN.cuda()

In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(sCNN.parameters(), lr=learning_rate)

In [40]:
num_epochs    = 10     # The number of times entire dataset is trained
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):   # Load a batch of images with its (index, data, class)
#         images = Variable(images.view(-1, 28*28))         # Convert torch tensor to Variable: change image from a vector of size 784 to a matrix of 28 x 28
        images = Variable(images)
        labels = Variable(labels)
#         print(images.shape)

        if use_cuda and torch.cuda.is_available():
            images = images.cuda()
            labels = labels.cuda()
        
        optimizer.zero_grad()                             # Intialize the hidden weight to all zeros
        outputs = sCNN(images)                             # Forward pass: compute the output class given a image
#         print(outputs.shape)
        loss = criterion(outputs, labels)                 # Compute the loss: difference between the output class and the pre-given label
        loss.backward()                                   # Backward pass: compute the weight
        optimizer.step()                                  # Optimizer: update the weights of hidden nodes
        
        if (i+1) % 600 == 0:                              # Logging
            print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))

Epoch [1/10], Step [600/600], Loss: 0.3074
Epoch [2/10], Step [600/600], Loss: 0.3336
Epoch [3/10], Step [600/600], Loss: 0.2549
Epoch [4/10], Step [600/600], Loss: 0.2247
Epoch [5/10], Step [600/600], Loss: 0.0931
Epoch [6/10], Step [600/600], Loss: 0.1012
Epoch [7/10], Step [600/600], Loss: 0.0753
Epoch [8/10], Step [600/600], Loss: 0.1323
Epoch [9/10], Step [600/600], Loss: 0.0417
Epoch [10/10], Step [600/600], Loss: 0.0804


**Testing on the test data**

In [41]:
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images)
    
    if use_cuda and torch.cuda.is_available():
        images = images.cuda()
        labels = labels.cuda()
    
    
    outputs = sCNN(images)
    _, predicted = torch.max(outputs.data, 1)  # Choose the best class from the output: The class with the best score
    total += labels.size(0)                    # Increment the total count
    correct += (predicted == labels).sum()     # Increment the correct count
    
print('Accuracy of the network on the 10K test images: %d %%' % (100 * correct / total))

Accuracy of the network on the 10K test images: 96 %


In [0]:
torch.save(sCNN.state_dict(), 'simpleCNN_model.pkl')

28